In [1]:
import pandas as pd

In [6]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
en=pd.read_csv('train.en',sep='\n',names=['Data'])
ne=pd.read_csv('train.ne',sep='\n',names=['Data'])



In [8]:
en.head()

,Data
0,"It happened after the death of Saul, when Davi..."
1,"it happened on the third day, that behold, a m..."
2,"David said to him, ""Where do you come from?"" H..."
3,"David said to him, ""How did it go? Please tell..."
4,"David said to the young man who told him, ""How..."


In [10]:
ne.head()

,Data
0,दाऊदले अमालेकीहरूलाई हराएर पछि सिकलग गए। यो शा...
1,तब तेस्रो दिनमा एउटा जवान सैनिक सिकलगमा आयो। त...
2,"दाऊदले त्यसलाई सोधे, “तिमी कहाँबाट आयौ?” त्यस ..."
3,"दाऊदले भने, “मलाई भन, के भयो?” त्यसले भन्यो, “..."
4,"दाऊदले त्यस सैनिकलाई भने, “तिमीले कसरी जान्यौ ..."


In [11]:
len(en)

139643

In [47]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [48]:
num_samples = 10000
num_samples

10000

In [49]:
for input_text,target_text in zip(en['Data'][:1000],ne['Data'][:1000]):
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [50]:
target_texts

['\tदाऊदले अमालेकीहरूलाई हराएर पछि सिकलग गए। यो शाऊलको मृत्यु भएको केही दिन पछिको कुरा हो। दाऊद त्यहाँ दुइ दिन बसे।\n',
 '\tतब तेस्रो दिनमा एउटा जवान सैनिक सिकलगमा आयो। त्यो मानिस शाऊलको छाउनीबाट आएको थियो। त्यसका लुगाहरू च्यतिएको र शिरमा मैला लागेको थियो। त्यसले दाऊदको अघि धोप्टो परेर उनलाई सम्मान गर्न दण्डवत् गर्यो।\n',
 '\tदाऊदले त्यसलाई सोधे, “तिमी कहाँबाट आयौ?” त्यस मानिसले जवाफ दियो, “म इस्राएली पालबाट आउँदैछु।”\n',
 '\tदाऊदले भने, “मलाई भन, के भयो?” त्यसले भन्यो, “हाम्रा सबै सैनिकहरू भागे। धेरै मानिसहरू मारिए। शाऊल र तिनका छोरा जोनाथन पनि मरे।”\n',
 '\tदाऊदले त्यस सैनिकलाई भने, “तिमीले कसरी जान्यौ शाऊल र जोनाथन मरेको कुरा?”\n',
 '\tत्यो सैनिकले उत्तर दियो, “म गिल्बो पर्वतमाथि थिएँ। मैले शाऊललाई आफ्नो भालामा झुकेको देखें। पलिश्ती सैनिकहरू घोडा र रथहरूमा चढेर शाऊलको नजिक आउँदै थिए।\n',
 '\tशाऊल वरिपरि घुमे अनि मलाई देखे र उनले मलाई बोलाए।\n',
 '\tतब शाऊलले मलाई को हो भनी सोधे। म अमालेकी हुँ भनि मैले उत्तर दिए।\n',
 '\tअनि शाऊलले भने, ‘म नराम्रो घाइते भएँ। दया गरी मलाई मारिदे। म अध

In [51]:
input_texts[99],target_texts[99]

('The king and his men went to Jerusalem against the Jebusites, the inhabitants of the land, who spoke to David, saying, "Unless you take away the blind and the lame, you shall not come in here"; thinking, "David can\'t come in here."',
 '\tतर पहिले एकपल्ट एउटाले सोच्यो यो समचार मेरो निम्ति असल थियो र भन्यो, उसले भन्यो, ‘हेर्नु होस् शाऊल मर्यो।’ उसले सोच्यो यो खबर सुनेर म उसलाई उचित पुरस्कार दिन्छु। तर मैले त्यसलाई सिकलगमा मारिदिएँ।\n')

In [52]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens: 64
Number of unique output tokens: 84
Max sequence length for inputs: 1173
Max sequence length for outputs: 415


In [53]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [54]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [55]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
          # decoder_target_data will be ahead by one timestep
          # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [56]:
import keras,tensorflow
from keras.models import Model
from keras.layers import Input,LSTM,Dense


In [57]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [58]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [59]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [60]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [61]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 64)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 84)     0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 256), (None, 328704      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 256),  349184      input_6[0][0]                    
                                                                 lstm_5[0][1]               

In [62]:
history=model.fit([encoder_input_data, decoder_input_data],
           decoder_target_data, batch_size=batch_size,
           epochs=epochs, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 13s 17ms/step - loss: 1.2991 - val_loss: 1.2901
Epoch 2/100
800/800 [==============================] - 13s 17ms/step - loss: 1.1949 - val_loss: 1.2852
Epoch 3/100
800/800 [==============================] - 14s 17ms/step - loss: 1.1886 - val_loss: 1.2809
Epoch 4/100
800/800 [==============================] - 14s 18ms/step - loss: 1.1794 - val_loss: 1.2674
Epoch 5/100
800/800 [==============================] - 14s 17ms/step - loss: 1.1684 - val_loss: 1.2541
Epoch 6/100
800/800 [==============================] - 14s 17ms/step - loss: 1.1539 - val_loss: 1.2352
Epoch 7/100
800/800 [==============================] - 14s 17ms/step - loss: 1.1324 - val_loss: 1.2325
Epoch 8/100
800/800 [==============================] - 15s 18ms/step - loss: 1.1114 - val_loss: 1.1810
Epoch 9/100
800/800 [==============================] - 15s 19ms/step - loss: 1.0809 - val_loss: 1.1534
Epoch 10/100
800/800 [=====

800/800 [==============================] - 14s 17ms/step - loss: 0.5333 - val_loss: 0.7031
Epoch 80/100
800/800 [==============================] - 13s 17ms/step - loss: 0.5309 - val_loss: 0.7057
Epoch 81/100
800/800 [==============================] - 14s 17ms/step - loss: 0.5273 - val_loss: 0.7141
Epoch 82/100
800/800 [==============================] - 13s 17ms/step - loss: 0.5242 - val_loss: 0.7011
Epoch 83/100
800/800 [==============================] - 14s 17ms/step - loss: 0.5223 - val_loss: 0.6999
Epoch 84/100
800/800 [==============================] - 14s 17ms/step - loss: 0.5191 - val_loss: 0.7056
Epoch 85/100
800/800 [==============================] - 14s 18ms/step - loss: 0.5170 - val_loss: 0.7060
Epoch 86/100
800/800 [==============================] - 14s 17ms/step - loss: 0.5127 - val_loss: 0.6968
Epoch 87/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5097 - val_loss: 0.7022
Epoch 88/100
800/800 [==============================] - 14s 17ms/step - loss:

In [63]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights()


encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)


# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encode the input sequence to get the internal state vectors.
    states_value = encoder_model.predict(input_seq)

    # generate empty target sequence of length 1 with only the start character
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # output sequence loop
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

        # sample a token and add the corresponding character to the 
        # decoded sequence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # check for the exit condition: either hitting max length
        # or predicting the 'stop' character
        if (sampled_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_length):
                stop_condition = True

        # update the target sequence (length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # update states
        states_value = [h, c]

    return decoded_sentence

In [64]:
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: It happened after the death of Saul, when David was returned from the slaughter of the Amalekites, and David had stayed two days in Ziklag;
Decoded sentence: तर दाऊदले भने, “म तिमीलाई यस्तो गर्नु भएको छ। म तिमीलाई योआबलाई भने, “म तिमीहरू तपाईंको सानुहरू पनि हान सान्नु भयो।

-
Input sentence: it happened on the third day, that behold, a man came out of the camp from Saul, with his clothes torn, and earth on his head: and so it was, when he came to David, that he fell to the earth, and showed respect.
Decoded sentence: तर दाऊदले भने, “म तिमीलाई यस्तो गर्नु भएको छ। म तिमीलाई योआबलाई भने, “म तिमीहरू तपाईंको सानुहरू पनि हान सान्नु भयो।

-
Input sentence: David said to him, "Where do you come from?" He said to him, "I have escaped out of the camp of Israel."
Decoded sentence: तर दाऊदले भने, “म तिमीलाई यस्तो गर्नु भएको छ। म तिमीलाई योआबलाई भने, “म तिमीहरू तपाईंको सानुहरू पनि हान सान्नु भयो।

-
Input sentence: David said to him, "How did it go? Please tell me." He answered, "

In [65]:
input_sentence = "How are you?"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
    test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

How are you?
तर दाऊदले भने, “म तिमीलाई यस्तो गर्नु भएको छ। म तिमीलाई योआबलाई भने, “म तिमीहरू तपाईंको सानुहरू पनि हान सान्नु भयो।



In [66]:
val_input_texts = []
val_target_texts = []
line_ix = 12000
for line in lines[line_ix:line_ix+10]:
    input_text, target_text = line.split('\t')
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)

val_encoder_input_data = np.zeros(
  (len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(val_input_texts):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.

NameError: name 'lines' is not defined

In [ ]:
for seq_index in range(10):
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', val_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence[:-1])
    print('Ground Truth sentence:', val_target_texts[seq_index])